In [3]:
import Pkg
Pkg.activate("..")
using ergm.models
using ergm.sampler
using ergm.spaces
using ergm.stats

  Activating project at `~/research/xaqlab/ergm.jl`
┌ Info: Precompiling ergm [caa93091-3dc3-4b2b-9430-1aae38e8753f]
└ @ Base loading.jl:1664


## Global Edge-Triangle Model

In [97]:
using LinearAlgebra
using Random
using Statistics

n = 100
node_positions = rand(n, 3)
local_radius = 100

function θ_to_E(θ)
    s = ParallelGibbsSampler(() -> LocalThreeNodeMotifStats(node_positions, local_radius), θ, 50000, 10, Threads.nthreads())
    ss = sample(s, 10000)
    Es = mean(ss, dims=2)
    Es
end

θ_to_E (generic function with 1 method)

In [80]:
ix = [(i, j) for i ∈ 1:n for j ∈ 1:n if i != j];
ix_l = filter(i -> norm(node_positions[i[1], :] - node_positions[i[2], :]) < local_radius, ix);
i_l = map(i -> i[1], ix_l)
j_l = map(i -> i[2], ix_l)
A = sparse(i_l, j_l, ones(Bool, length(ix_l)))
sum(A ^ 2 .* A) / 6

8027.0

In [81]:
length(ix_l) / length(ix) * 100

27.494949494949495

In [82]:
n * (n - 1) * (n - 2) / 6

161700.0

In [83]:
8027/161700 * 100

4.964131106988249

In [98]:
using DataFrames
using JLD

df_global = DataFrame(θe = Float64[], θt = Float64[], θt_c = Int64[], Ee = Float64[], Et = Float64[], Et_ER = Float64[])
θes = [sign(p) * 10^abs(p) for p ∈ -5:0.25:5]
θts = [sign(p) * 10^abs(p) for p ∈ -5:1:5]

for θe ∈ θes
    for (θt_c, θt) ∈ enumerate(θts)
        θ = [θe, θt]
        println(θ)
        flush(stdout)
        Ee, Et = θ_to_E(θ)
        push!(df_global, (θe, θt, θt_c, Ee, Et, Ee ^ 3))
    end
end

[-100000.0, -100000.0]
[-100000.0, -10000.0]
[-100000.0, -1000.0]
[-100000.0, -100.0]
[-100000.0, -10.0]
[-100000.0, 0.0]
[-100000.0, 10.0]
[-100000.0, 100.0]
[-100000.0, 1000.0]
[-100000.0, 10000.0]
[-100000.0, 100000.0]
[-56234.13251903491, -100000.0]
[-56234.13251903491, -10000.0]
[-56234.13251903491, -1000.0]
[-56234.13251903491, -100.0]
[-56234.13251903491, -10.0]
[-56234.13251903491, 0.0]
[-56234.13251903491, 10.0]
[-56234.13251903491, 100.0]
[-56234.13251903491, 1000.0]
[-56234.13251903491, 10000.0]
[-56234.13251903491, 100000.0]
[-31622.776601683792, -100000.0]
[-31622.776601683792, -10000.0]
[-31622.776601683792, -1000.0]
[-31622.776601683792, -100.0]
[-31622.776601683792, -10.0]
[-31622.776601683792, 0.0]
[-31622.776601683792, 10.0]
[-31622.776601683792, 100.0]
[-31622.776601683792, 1000.0]
[-31622.776601683792, 10000.0]
[-31622.776601683792, 100000.0]
[-17782.794100389227, -100000.0]
[-17782.794100389227, -10000.0]
[-17782.794100389227, -1000.0]
[-17782.794100389227, -100.0]

In [100]:
using Gadfly

Gadfly.push_theme(:default)
p = plot(
    layer(df_global, x=:Ee, y=:Et, color=:θt_c),
    layer(df_global, x=:Ee, y=map(Ee -> Ee ^ 3, df_global[:, :Ee]), Geom.line, linestyle=[:dash]),
    Guide.xlabel("edge density"),
    Guide.ylabel("triangle density"),
    Guide.title("Classic edge-triangle ERGM (100 nodes, 100% edges local)"),
    Theme(key_position = :none)
)
img = SVG("classic_edge_triangle.svg", 5inch, 5inch)
draw(img, p)

false

In [319]:
ix = 0.6 .< df_global[:, :Ee] .< 0.8
df_global[ix, [:θe, :θt, :Et]]

,θe,θt,Et
,Float64,Float64,Float64
1,-316.228,3162.28,0.966413
2,-100.0,1000.0,0.580464
3,-10.0,1000.0,0.592686
4,316.228,100.0,0.210756
5,316.228,316.228,0.2983
6,1000.0,-316.228,0.326309
7,3162.28,-3162.28,0.209889
8,10000.0,-10000.0,0.184465


In [332]:
Es = θ_to_E([-316, 3162])

2×1 Matrix{Float64}:
 0.8877022857142487
 0.9675855255100853

In [333]:
Es[1]^3

0.6995230256528144

## Local Edge-Triangle Model

In [271]:
local_radius = 10
θ = [1e3, 1e3]
s = ParallelGibbsSampler(() -> stats.EdgeLocalTriangleStats(node_positions, local_radius), θ, 10000, 3000, Threads.nthreads())
ss = sample(s, 10)
Es = mean(ss, dims=2)

2×1 Matrix{Float64}:
 0.8075102040816328
 0.0

In [264]:
a =  stats.EdgeLocalTriangleStats(node_positions, local_radius)

ergm.stats.EdgeLocalTriangleStats(Graph(Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 50, 0), [0.14593849008483128 0.4044296905569703 … 0.9681375387641071 0.5099639384972953; 0.8546514952788975 0.23839460181351746 … 0.33528614989098704 0.5763473560550354; 0.5557289516504796 0.023439291247570604 … 0.3143167353079489 0.26387036961446375], 10, Graph(Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 50, 0), 0, 0, 1225, 117600, Dict{Any, Any}())